<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [135]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    #strip removes leading/trailing white space date_time column
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

# def booster_version(table_cells):
#     """
#     This function returns the booster version from the HTML  table cell 
#     Input: the  element of a table data cell extracts extra row
#     """
#     out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
#     return out

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML table cell.
    Input: The element of a table data cell, extracts the booster version.
    """
    # Extract the strings from the table cells
    booster_versions = [booster_version for booster_version in table_cells.strings]
    
    # Join the strings, excluding the last one if it's an unwanted entry
    out = ''.join(booster_versions[:-1])  # Exclude the last element if needed
     # Keep only the first 2 characters of the resulting string
    return out[:2]  # Slicing to keep only the first 2 characters
    

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [45]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [15]:
# use requests.get() method with the provided static_url
# assign the response to a object

# Send a GET request to the URL and assign the response to an object
#response = requests.get(static_url)

# Optionally, check the status of the response (e.g., 200 for success)
# ?if response.status_code == 200:
#     print("Request was successful.")
# else:
#     print(f"Request failed with status code: {response.status_code}")

# Access the content of the response (if JSON, for example)
#data = response.json()  # If the response is in JSON format

# Display the data
#print(data)

Request was successful.


Create a `BeautifulSoup` object from the HTML `response`


In [ ]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
# Define the URL
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"  # Replace with your actual URL

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Create a BeautifulSoup object from the HTML response
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Print the parsed HTML (or you can work with the `soup` object)
    print(soup.prettify())
else:
    print(f"Request failed with status code: {response.status_code}")

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [49]:
# Use soup.title attribute
 # Access the <title> tag using the soup.title attribute
page_title = soup.title
# Print the title and its text content
print("Title tag:", page_title)
print("Title text:", page_title.string)  # To get just the text inside the <title> tag
# else:
# print(f"Request failed with status code: {response.status_code}")

Title tag: <title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
Title text: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [55]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
 # Use find_all to extract all <table> elements and assign them to a list called html_tables
html_tables = soup.find_all('table')

# Print the number of tables found
print(f"Number of tables found: {len(html_tables)}")

# Optionally, you can loop through and print each table
# for index, table in enumerate(html_tables):
#     print(f"\nTable {index + 1}:")
#     print(table.prettify())  # Prettify the HTML for better readability


Number of tables found: 26


Starting from the third table is our target table contains the actual launch records.


In [ ]:
first_launch_table = html_tables[2]
print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [65]:
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

# Initialize an empty list to store column names
column_names = []

# Apply the find_all() function to get all <th> elements in first_launch_table
table_headers = first_launch_table.find_all('th')

# Initialize a dictionary to store the length of each column
# column_lengths = {}

# Iterate over each <th> element and extract the column name
for th in table_headers:
    # Get the text content of the <th> element and strip any leading/trailing whitespace
    column_name = th.get_text(strip=True)
    
    # Append the column name to the list if it is not None and non-empty
    if column_name is not None and len(column_name) > 0:
        column_names.append(column_name)
        # column_lengths[column_name] = 0  # Initialize length to 0

# Print the column names
print(column_names)


['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [67]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date andtime (UTC)'] #updated
del launch_dict['1']
del launch_dict['2']
del launch_dict['3']
del launch_dict['4']
del launch_dict['5']
del launch_dict['6']
del launch_dict['7']
del launch_dict['Version,Booster[b]']
del launch_dict['Boosterlanding']

#rename to fit the following- parsing was crap
column_names[column_names.index('Payload[c]')] = 'Payload'


# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launchoutcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [69]:
print(launch_dict)

{'Flight No.': [], 'Launch site': [], 'Payload[c]': None, 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launchoutcome': [], 'Payload': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}


In [71]:
del launch_dict['Payload[c]']
print(launch_dict)

{'Flight No.': [], 'Launch site': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launchoutcome': [], 'Payload': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [200]:
#DONT" USE DOESN"T LIMIT ROWS OD DATA_ INCORRECT
# Initialize the launch_dict to store data
# launch_dict = {'Flight No.': [], 'Launch site': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launchoutcome': [], 'Payload': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}
# trying to troubleshoot why so many entires... should only be 121 rows
# Counter for extracted rows
# extracted_row = 0

# # Extract each table
# for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
#     # Get table rows
#     for rows in table.find_all("tr"):
#         # Check if first table heading corresponds to a launch number
#         if rows.th:
#             if rows.th.string:
#                 flight_number = rows.th.string.strip()
#                 flag = flight_number.isdigit()  # Ensure it's a valid flight number
#         else:
#             flag = False

#         # Get table element (td elements)
#         row = rows.find_all('td')

#         # If it is a number, save cells in launch_dict
#         if flag:
#             extracted_row += 1

#             # Extract data and append to the corresponding lists in launch_dict

#             # Flight Number
#             launch_dict['Flight No.'].append(flight_number)

#             # Get date and time from the first cell
#             datatimelist = date_time(row[0])
#             date = datatimelist[0].strip(',')
#             time = datatimelist[1]

#             # Date and Time
#             launch_dict['Date'].append(date)
#             launch_dict['Time'].append(time)

#             # Version Booster
#             bv = booster_version(row[1])
#             launch_dict['Version Booster'].append(bv)

#             # Launch Site
#             if row[2].a:
#                 launch_site = row[2].a.string
#             else:
#                 launch_site = None
#             launch_dict['Launch site'].append(launch_site)

#             # Payload
#             if row[3].a:
#                 payload = row[3].a.string
#             else:
#                 payload = None
#             launch_dict['Payload'].append(payload)

#             # Payload Mass
#             payload_mass = get_mass(row[4])
#             launch_dict['Payload mass'].append(payload_mass)

#             # Orbit
#             if row[5].a:
#                 orbit = row[5].a.string
#             else:
#                 orbit = None
#             launch_dict['Orbit'].append(orbit)

#             # Customer
#             if row[6].a:
#                 customer = row[6].a.string
#             else:
#                 customer = None
#             launch_dict['Customer'].append(customer)

#             # Launch Outcome
#             if row[7].strings:
#                 launch_outcome = list(row[7].strings)[0]
#             else:
#                 launch_outcome = None
#             launch_dict['Launchoutcome'].append(launch_outcome)

#             # Booster Landing
#             booster_landing = landing_status(row[8])
#             launch_dict['Booster landing'].append(booster_landing)

# # Print the number of entries for each field
# for key, value in launch_dict.items():
#     print(f"{key}: {len(value)} entries")
#all of the entries have diffrent numbers

Flight No.: 715 entries
Launch site: 471 entries
Payload mass: 470 entries
Orbit: 470 entries
Customer: 469 entries
Launchoutcome: 468 entries
Payload: 471 entries
Version Booster: 472 entries
Booster landing: 468 entries
Date: 363 entries
Time: 363 entries


In [202]:
#use this- limits to past launches - table format changes after flight 121

# Initialize the launch_dict to store data
launch_dict = {'Flight No.': [], 'Launch site': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launchoutcome': [], 'Payload': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}

# Counter for extracted rows
extracted_row = 0
max_entries = 121  # Limit to the first 121 entries

# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table rows
    for rows in table.find_all("tr"):
        # Stop extracting if we reach the limit of 121 entries
        if extracted_row >= max_entries:
            break

        # Check if first table heading corresponds to a launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()  # Ensure it's a valid flight number
        else:
            flag = False

        # Get table element (td elements)
        row = rows.find_all('td')

        # If it is a number, save cells in launch_dict
        if flag:
            extracted_row += 1  # Increment row counter

            # Extract data and append to the corresponding lists in launch_dict

            # Flight Number
            launch_dict['Flight No.'].append(flight_number)

            # Get date and time from the first cell
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]

            # Date and Time
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Version Booster
            bv = booster_version(row[1])
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            if row[2].a:
                launch_site = row[2].a.string
            else:
                launch_site = None
            launch_dict['Launch site'].append(launch_site)

            # Payload
            if row[3].a:
                payload = row[3].a.string
            else:
                payload = None
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            if row[5].a:
                orbit = row[5].a.string
            else:
                orbit = None
            launch_dict['Orbit'].append(orbit)

            # Customer
            if row[6].a:
                customer = row[6].a.string
            else:
                customer = None
            launch_dict['Customer'].append(customer)

            # Launch Outcome
            if row[7].strings:
                launch_outcome = list(row[7].strings)[0]
            else:
                launch_outcome = None
            launch_dict['Launchoutcome'].append(launch_outcome)

            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Print the number of entries for each field after processing the first 121 rows
for key, value in launch_dict.items():
    print(f"{key}: {len(value)} entries")


Flight No.: 121 entries
Launch site: 121 entries
Payload mass: 121 entries
Orbit: 121 entries
Customer: 121 entries
Launchoutcome: 121 entries
Payload: 121 entries
Version Booster: 121 entries
Booster landing: 121 entries
Date: 121 entries
Time: 121 entries


In [ ]:
#DONT USE DOESN"T LIMIT ENTRIES_ USE CODE ABOVE
# #probelm as table chage after flight 103??- requires some work...
# extracted_row = 0
# #Extract each table 
# for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
#    # get table row 
#     for rows in table.find_all("tr"):
#         #check to see if first table heading is as number corresponding to launch a number 
#         if rows.th:
#             if rows.th.string:
#                 flight_number=rows.th.string.strip()
#                 flag=flight_number.isdigit()
#         else:
#             flag=False
       
#         #get table element 
#         row=rows.find_all('td')
       
#         #if it is number save cells in a dictonary 
#         if flag:
#             extracted_row += 1
           
#             # Flight Number value
#             # TODO: Append the flight_number into launch_dict with key `Flight No.`
#             launch_dict['Flight No.'].append(flight_number)
#             # print(flight_number)
            
#             # Get date and time from the first cell
#             datatimelist=date_time(row[0])
#             # print(datatimelist)
            
#             # # Date value
#             # # TODO: Append the date into launch_dict with key `Date`
#             date = datatimelist[0].strip(',')
#             # print(date)
#             launch_dict['Date'].append(date)

            
#             # # Time value
#             # # TODO: Append the time into launch_dict with key `Time`
#             time = datatimelist[1]
#             # print(time)
#             launch_dict['Time'].append(time)  # Combine date and time
              
#             # Booster version
#             # TODO: Append the bv into launch_dict with key `Version Booster`
#             bv=booster_version(row[1])
#             # print(bv)                
#             launch_dict['Version Booster'].append(bv)
            
            
#             # # Launch Site
#             # # TODO: Append the launch_site into launch_dict with key `Launch Site
#             #a.sting looks for first anchor tag string description`
#             if row[2].a:  # Check if there's an anchor tag
#                  launch_site = row[2].a.string
#             else:
#                  launch_site = None  # Fallback if no anchor tag exists
#             # print(launch_site)
#             launch_dict['Launch site'].append(launch_site)
           
            
#             # # Payload
#             # # TODO: Append the payload into launch_dict with key `Payload`
#             if row[3].a:
#                payload = row[3].a.string  
#             else:
#                payload = None  # Fallback if no anchor tag exists
#             # print(payload)
#             launch_dict['Payload'].append(payload)
             
            
#             # # Payload Mass
#             # # TODO: Append the payload_mass into launch_dict with key `Payload mass`
#             payload_mass = get_mass(row[4]) 
#             # print(payload_mass)
#             launch_dict['Payload mass'].append(payload_mass)
            
            
#             # # Orbit
#             # # TODO: Append the orbit into launch_dict with key `Orbit`
#             if row[5].a:
#                 orbit = row[5].a.string
#             else:
#                orbit = None  # Fallback if no anchor tag exists
#             # print(orbit)
#             launch_dict['Orbit'].append(orbit)
            
            
#             # # Customer
#             # # TODO: Append the customer into launch_dict with key `Customer`
#             if row[6].a:
#                 customer = row[6].a.string   
#             else:
#                orbit = None  # Fallback if no anchor tag exists
#             # print(customer)
#             launch_dict['Customer'].append(customer)
           
            
#             # Launch outcome
#             # # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
#             if row[7].strings:
#                 launch_outcome = list(row[7].strings)[0]
#             else:
#                 launch_outcome = None  # Handle if no strings
#             # print(launch_outcome)
#             launch_dict['Launchoutcome'].append(launch_outcome)
           
            
#             # # Booster landing
#             # # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
#             booster_landing = landing_status(row[8])
#             # print(booster_landing)
#             launch_dict['Booster landing'].append(booster_landing)
           
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [204]:
for key, value in launch_dict.items():
    print(f"{key}: {len(value)}")


Flight No.: 121
Launch site: 121
Payload mass: 121
Orbit: 121
Customer: 121
Launchoutcome: 121
Payload: 121
Version Booster: 121
Booster landing: 121
Date: 121
Time: 121


In [206]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [208]:
df.head(15)

,Flight No.,Launch site,Payload mass,Orbit,Customer,Launchoutcome,Payload,Version Booster,Booster landing,Date,Time
0,1,CCAFS,0,LEO,SpaceX,Success\n,Dragon Spacecraft Qualification Unit,F9,Failure,4 June 2010,18:45
1,2,CCAFS,0,LEO,NASA,Success,Dragon,F9,Failure,8 December 2010,15:43
2,3,CCAFS,525 kg,LEO,NASA,Success,Dragon,F9,No attempt\n,22 May 2012,07:44
3,4,CCAFS,"4,700 kg",LEO,NASA,Success\n,SpaceX CRS-1,F9,No attempt,8 October 2012,00:35
4,5,CCAFS,"4,877 kg",LEO,NASA,Success\n,SpaceX CRS-2,F9,No attempt\n,1 March 2013,15:10
5,6,VAFB,500 kg,Polar orbit,MDA,Success,CASSIOPE,F9,Uncontrolled,29 September 2013,16:00
6,7,CCAFS,"3,170 kg",GTO,SES,Success,SES-8,F9,No attempt,3 December 2013,22:41
7,8,CCAFS,"3,325 kg",GTO,Thaicom,Success,Thaicom 6,F9,No attempt,6 January 2014,22:06
8,9,Cape Canaveral,"2,296 kg",LEO,NASA,Success\n,SpaceX CRS-3,F9,Controlled,18 April 2014,19:25
9,10,Cape Canaveral,"1,316 kg",LEO,Orbcomm,Success,Orbcomm-OG2,F9,Controlled,14 July 2014,15:15


In [210]:
df.tail()

,Flight No.,Launch site,Payload mass,Orbit,Customer,Launchoutcome,Payload,Version Booster,Booster landing,Date,Time
116,117,CCSFS,"15,600 kg",LEO,SpaceX,Success\n,Starlink,F9,Success,9 May 2021,06:42
117,118,KSC,"~14,000 kg",LEO,SpaceX,Success\n,Starlink,F9,Success,15 May 2021,22:56
118,119,CCSFS,"15,600 kg",LEO,SpaceX,Success\n,Starlink,F9,Success,26 May 2021,18:59
119,120,KSC,"3,328 kg",LEO,NASA,Success\n,SpaceX CRS-22,F9,Success,3 June 2021,17:29
120,121,CCSFS,"7,000 kg",GTO,Sirius XM,Success\n,SXM-8,F9,Success,6 June 2021,04:26


In [212]:
#original file download
# filepath ='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'
# df_original = pd.read_json(filepath)
file_path = r"C:\Users\Raegan\Downloads\AppliedDataScinece_SpaceXProj_Webscraped_Part2.csv"  #remname file
try:
    df.to_csv(file_path, index=False)
    print(f'DataFrame saved to {file_path}')
except Exception as e:
    print(f'An error occurred: {e}')

DataFrame saved to C:\Users\Raegan\Downloads\AppliedDataScinece_SpaceXProj_Webscraped_Part2.csv


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
